<a href="https://colab.research.google.com/github/enesemretas/mcpath-colab/blob/main/notebooks/mcpath_form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title Launch MCPath UI from repo (robust import)
!pip -q install pyyaml ipywidgets requests

import sys, pathlib, shutil, subprocess, os, importlib

REPO_URL = "https://github.com/enesemretas/mcpath-colab.git"
REPO_DIR = pathlib.Path("/content/mcpath-colab")
PKG_DIR  = REPO_DIR / "mcpath"

# Clone if needed; otherwise pull latest
if not REPO_DIR.exists():
    subprocess.run(["git", "clone", "--depth", "1", REPO_URL, str(REPO_DIR)], check=True)
else:
    subprocess.run(["git", "-C", str(REPO_DIR), "pull"], check=True)

# Ensure package is a proper Python package
PKG_DIR.mkdir(parents=True, exist_ok=True)
init_file = PKG_DIR / "__init__.py"
if not init_file.exists():
    init_file.write_text("")  # create empty __init__.py if missing

# Make the repo importable
if str(REPO_DIR) not in sys.path:
    sys.path.append(str(REPO_DIR))

# Optional: print a quick tree for sanity
print("Repo contents:", list(REPO_DIR.iterdir()))
print("mcpath contents:", list(PKG_DIR.iterdir()))

# Import & launch
import mcpath.ui as mcui
importlib.reload(mcui)  # in case you re-run this cell after edits
from mcpath.ui import launch

# For ipywidgets in Colab
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except Exception:
    pass

launch(
    defaults_url="https://raw.githubusercontent.com/enesemretas/mcpath-colab/main/config/defaults.yaml",
    show_title="MCPath (Python readpdb)"
)


ModuleNotFoundError: No module named 'mcpath'